# Use HYPER_PERSNL_Template Customized Notebook Template

In [2]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
infer_datetime_format=True

import numpy as np
import random

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

random.seed(0)

now1 = datetime.now()

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [4]:
table_name = 'CUSTOMER_ACCOUNT_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
account = sf_df.to_pandas()

In [6]:
table_name = 'CUSTOMER_TRANSACTION_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
transactions = sf_df.to_pandas()

In [7]:
transactions.columns

Index(['ACC_BBAN', 'ACC_IBAN', 'ACC_OPEN_DT', 'BALANCE', 'CUST_ID', 'EXPENSE',
       'EXPENSE_CATEGORY', 'BROWSER', 'DEVICE', 'DR_CR', 'LOGIN_TIME',
       'LOGOUT_TIME', 'TXN_DT', 'TXN_ID', 'TXN_AMOUNT', 'TXN_CODE',
       'TXN_REMARKS', 'TXN_TIME', 'TXN_TYPE'],
      dtype='object')

In [8]:
transactions.sort_values(by=['CUST_ID', 'ACC_BBAN', 'TXN_DT', 'TXN_TIME'], inplace=True) 

In [9]:
table_name = 'CUSTOMER_FINANCE_BREAKDOWN'

sf_df = my_session.sql("select * from {}".format(table_name))
financials = sf_df.to_pandas()

In [10]:
table_name = 'CUSTOMER_SUBSCRIPTION_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
subscriptions = sf_df.to_pandas()
subscriptions_orig = subscriptions.copy(deep=True)

In [11]:
table_name = 'CUSTOMER_TERM_DEPOSIT_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
term_deposit = sf_df.to_pandas()

In [12]:
table_name = 'CUSTOMER_LOAN_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
loan = sf_df.to_pandas()

In [13]:
table_name = 'CUSTOMER_CREDITCARD_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
credit_card = sf_df.to_pandas()

In [15]:
x1 = list(set(transactions['CUST_ID']))

In [19]:
account

,CUST_ID,ACC_IBAN,ACC_BBAN,ACC_OPEN_DT,BALANCE,ACC_STATUS,ACC_TYPE
0,4185,GB90EMIC89482763544524,GDJY66571183273954,2001-03-22,$50.57,Active,Credit
1,291,GB83VYSJ52094207716710,OVWN34297715684777,2010-08-29,"$74,111.13",Closed,Current
2,7497,GB17OCSO08226852523762,DLFO12501444293678,2000-09-04,$669.14,Active,Loan
3,7497,GB26RRMJ41734703010562,SZPN32742192244415,2010-09-05,$0.03,Active,Credit
4,7497,GB13JBDZ14742645466984,DAGC95804976124339,2006-02-14,$0.71,Closed,Credit
...,...,...,...,...,...,...,...
3101,2137,GB79MEEM68164816755595,JGQZ63653199537639,2020-07-08,$191.20,Active,Savings
3102,2137,GB72FWKV94464237567600,LCGK26173403352193,2011-06-05,"$54,340.79",Closed,Savings
3103,3899,GB97XFCA41641435894700,IQWX43085486211593,2006-01-08,$906.56,Active,Loan
3104,3899,GB72HHTA49364267690196,PFVP16819625415314,2014-10-14,$17.61,Closed,Loan


In [22]:
#bank_name = list(set(account['BANK_NAME'])) --> Unable to find Bank_Name in account table.
bank_name = ['UBS',
'Bank of America',
'Discover Financial',
'Credit Suisse',
'Santander Bank',
'American Express',
'HSBC Bank USA',
'JPMorgan Chase',
'BankUnited',
'Citigroup',
'Barclays',
'RBC Bank',
'Citizens Financial Group',
'Capital One',
'Goldman Sachs',
'Wells Fargo',
'Morgan Stanley']

# Reversing a list using slicing technique

In [21]:
def Reverse(lst):
    new_lst = lst[::-1]
    return new_lst

# Initiate the DataFrames 

In [23]:
EOD_Acc_Level = pd.DataFrame(columns = ['CUST_ID', 'ACC_IBAN', 'ACC_BBAN', 'DT', 'EOD_BALANCE', 
                                  'EOD_TOTAL_CR', 'EOD_TOTAL_DR', 'EOD_TOTAL_INCOME', 
                                  'EOD_TOTAL_EXPENSE'])

In [24]:
EOD_Cust_Level = pd.DataFrame(columns = ['CUST_ID', 'DT', 'EOD_BALANCE', 'EOD_TOTAL_CR', 'EOD_TOTAL_DR', 
                                  'EOD_TOTAL_INCOME', 'EOD_TOTAL_BALANCE'])

In [25]:
EOW_Acc_Level = pd.DataFrame(columns = ['CUST_ID', 'ACC_IBAN', 'ACC_BBAN', 'WK_NUM-YR', 'WK_START_DT', 'Wk_END_DT', 'EOW_BALANCE', 'EOW_TOTAL_DR', 
                                        'EOW_TOTAL_CR','EOW_TOTAL_INCOME', 'EOW_TOTAL_EXPENSE', 'SLOPE_BALANCE',
                              'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR', 'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [26]:
EOW_Cust_Level = pd.DataFrame(columns = ['CUST_ID', 'WK_NUM-YR', 'WK_START_DT', 'Wk_END_DT', 'EOW_BALANCE',
       'EOW_TOTAL_CR', 'EOW_TOTAL_DR', 'EOW_TOTAL_INCOME', 'EOW_TOTAL_EXPENSE','SLOPE_BALANCE', 
       'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR', 'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [27]:
EOM_Acc_Level = pd.DataFrame(columns = ['CUST_ID', 'ACC_IBAN', 'ACC_BBAN', 'MTH_NUM-YR', 'EOM_BALANCE',
       'EOM_TOTAL_DR', 'EOM_TOTAL_CR', 'EOM_TOTAL_INCOME', 'EOM_TOTAL_EXPENSE', 'SLOPE_BALANCE', 
       'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR',  'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [28]:
EOM_Cust_Level = pd.DataFrame(columns = ['CUST_ID', 'MTH_NUM-YR', 'EOM_BALANCE', 'EOM_TOTAL_CR', 
       'EOM_TOTAL_DR', 'EOM_TOTAL_INCOME', 'EOM_TOTAL_EXPENSE', 'SLOPE_BALANCE',
       'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR',  'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [29]:
Cust_Dashboard = pd.DataFrame(columns = ['CUST_ID', 'BALANCE', 'AVG_BALANCE', 'AVG_INCOME', 'AVG_EXPENSE', 'BANK_NAME'])   

# Aggregation logic for Daily, Weekly and Monthly 

In [95]:
for i in x1: ### This is coming from account table
    ibans = list(set(transactions[transactions['CUST_ID']==i]['ACC_IBAN']))
    for j in ibans:  ### for a given customer pickup one IBAN at a time
        txn_temp = transactions.loc[(transactions['CUST_ID']==i) & (transactions['ACC_IBAN']==j), ['ACC_BBAN', 'ACC_IBAN','BALANCE', 'TXN_DT', 'DR_CR', 'TXN_AMOUNT', 'TXN_CODE']]    ### This is at account level
        txn_Dr = txn_temp.loc[(txn_temp['DR_CR']=='Dr')]    ### This is at account level
        txn_Cr = txn_temp.loc[(txn_temp['DR_CR']=='Cr')]    ### This is at account level
        ##txn_temp['txn_Dt'] = txn_temp['txn_Dt'].astype(str)

        #### Process as per balance column for EOD Balance ##############
        # Dts = txn_temp['TXN_DT'].drop_duplicates() -> Giving Error as m/d/yyyy
        Dts = pd.to_datetime(txn_temp['TXN_DT'].drop_duplicates(), format='%m/%d/%Y')

        sdate = datetime.strptime(str(min(Dts))[:10], '%Y-%m-%d')
        edate = datetime.strptime(str(max(Dts))[:10], '%Y-%m-%d')        
        
        dt_range = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
        hn = len(dt_range)
        eod_balance = []

        for di in Dts:
            try:
                eod_balance = eod_balance + [txn_temp.loc[txn_temp['TXN_DT']==di, ['TXN_DT', 'BALANCE']].iloc[-1, 1]]
            except:
                print(di)
        temp_Acc_Daily = pd.DataFrame({'DT': list(Dts), 'EOD_BALANCE': eod_balance})
        temp_Acc_Daily.reset_index(inplace=True) 
        temp_Acc_Daily['DT']=temp_Acc_Daily['DT'].astype('str')
        delta_Acc_Daily = pd.DataFrame({'CUST_ID': [i]*hn, 'ACC_IBAN': [j]*hn, 'ACC_BBAN':[txn_temp.iloc[0,0]]*hn, 
                                        'DT': dt_range})
        delta_Acc_Daily['DT']=delta_Acc_Daily['DT'].astype('str')
        #delta_Acc_Daily = pd.merge(delta_Acc_Daily, temp_Acc_Daily, on='DT', how='left')
        
        ## delta_Acc_Daily.to_csv('delta_Acc_Daily.csv', index=False)
        #delta_Acc_Daily.drop('index', axis=1, inplace=True)
        #gt0_values = delta_Acc_Daily.loc[delta_Acc_Daily['EOD_BALANCE']>0, 'EOD_BALANCE']
        #gt0_idx = list(gt0_values.index) + [len(delta_Acc_Daily)]
        #eod_balance=[]
        #for idx in range(len(gt0_idx)-1):
        #    eod_balance = eod_balance+[gt0_values[gt0_idx[idx]]]*(gt0_idx[idx+1]-gt0_idx[idx])
        #delta_Acc_Daily['EOD_BALANCE'] = eod_balance



2021-01-01 00:00:00
2018-01-10 00:00:00
2018-01-11 00:00:00
2021-01-11 00:00:00
2019-01-16 00:00:00
2018-01-17 00:00:00
2018-01-19 00:00:00
2018-01-20 00:00:00
2017-01-22 00:00:00
2018-01-23 00:00:00
2021-01-23 00:00:00
2020-01-26 00:00:00
2021-01-26 00:00:00
2019-01-27 00:00:00
2019-01-28 00:00:00
2019-01-29 00:00:00
2018-01-03 00:00:00
2020-01-30 00:00:00
2021-01-30 00:00:00
2017-01-06 00:00:00
2020-01-06 00:00:00
2019-01-07 00:00:00
2020-01-07 00:00:00
2020-01-08 00:00:00
2017-10-01 00:00:00
2017-10-10 00:00:00
2020-10-10 00:00:00
2016-10-11 00:00:00
2020-10-12 00:00:00
2021-10-13 00:00:00
2016-10-15 00:00:00
2019-10-17 00:00:00
2017-10-18 00:00:00
2019-10-19 00:00:00
2020-10-19 00:00:00
2016-10-02 00:00:00
2018-10-02 00:00:00
2020-10-02 00:00:00
2021-10-02 00:00:00
2017-10-21 00:00:00
2018-10-21 00:00:00
2020-10-21 00:00:00
2016-10-22 00:00:00
2017-10-22 00:00:00
2019-10-22 00:00:00
2019-10-24 00:00:00
2016-10-26 00:00:00
2019-10-27 00:00:00
2020-10-27 00:00:00
2017-10-04 00:00:00


ValueError: All arrays must be of the same length